# Leverage-etf-Strategy Interactive Brokers execution

In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import ibapi
import pandas as pd

import threading
import time

from datetime import datetime, timedelta

print(pd.__version__)
print(ibapi.__version__)

ticker = 'MNQ'

yesterday = str(datetime.now() - timedelta(1))
yesterday = yesterday.replace('-', '')
date = '{}-18:00:00'.format(yesterday[:8])

save = '{}-{}-{}-{}.csv'.format(ticker, date[2:4], date[4:6], date[6:8])

contract_month = "202212"
multiplier = 2
exchange = 'CME'
security = 'FUT'
frequency = '1 day'
interval = '1 W'

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = [] #Initialize variable to store candle

    def historicalData(self, reqId, bar):
        print(f'Time: {bar.date} Open: {bar.open} High: {bar.high} Low: {bar.low} Close: {bar.close}')
        self.data.append([bar.date, bar.open, bar.high, bar.low, bar.close])
        
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 125)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
def futures_contract(symbol, secType, exchange, contract_month, multiplier):
    ''' custom function to create contract '''
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.lastTradeDateOrContractMonth = contract_month
    contract.multiplier = multiplier
    return contract


NQ_contract = futures_contract(ticker, security, exchange, contract_month, multiplier)


#Request historical candles
app.reqHistoricalData(1, NQ_contract, date, interval, frequency, 'TRADES', 0, 2, False, [])
time.sleep(5) #sleep to allow enough time for data to be returned

#Working with Pandas DataFrames
import pandas

df = pandas.DataFrame(app.data, columns=['DateTime', 'Open', 'High', 'Low', 'Close'])

df.head()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


1.2.4
9.81.1
Time: 20221201 Open: 12079.25 High: 12145.0 Low: 11943.25 Close: 12062.75
Time: 20221202 Open: 12039.75 High: 12067.75 Low: 11740.5 Close: 12010.25
Time: 20221205 Open: 12000.5 High: 12015.75 Low: 11745.5 Close: 11805.75
Time: 20221206 Open: 11812.0 High: 11846.25 Low: 11502.0 Close: 11566.0
Time: 20221207 Open: 11566.0 High: 11596.25 Low: 11422.5 Close: 11509.5


,DateTime,Open,High,Low,Close
0,20221201,12079.25,12145.00,11943.25,12062.75
1,20221202,12039.75,12067.75,11740.50,12010.25
2,20221205,12000.50,12015.75,11745.50,11805.75
3,20221206,11812.00,11846.25,11502.00,11566.00
4,20221207,11566.00,11596.25,11422.50,11509.50


In [2]:
prior_close = df.iloc[-1,-1]
prior_close_long = (prior_close * 1.00667)
prior_close_short = (prior_close * 0.99333)
print(prior_close)
print(prior_close_long)
print(prior_close_short)

11509.5
11586.268365
11432.731635


In [3]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import pandas as pd
import threading
import time
import datetime


class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.bardata = {} #Initialize dictionary to store bar data

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def tick_df(self, reqId, contract):
        ''' custom function to init DataFrame and request Tick Data '''
        self.bardata[reqId] = pd.DataFrame(columns=['time', 'price'])
        self.bardata[reqId].set_index('time', inplace=True)
        self.reqTickByTickData(reqId, contract, "Last", 0, True)
        return self.bardata[reqId]

    def tickByTickAllLast(self, reqId, tickType, time, price, size, tickAtrribLast, exchange, specialConditions):
        if tickType == 1:
            self.bardata[reqId].loc[pd.to_datetime(time, unit='s')] = price

    def futures_contract(self, symbol, secType='FUT', exchange='CME'):
        ''' custom function to create contract '''
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.lastTradeDateOrContractMonth = "202212"
        contract.multiplier = 2
        return contract


def run_loop():
    app.run()

def submit_order(contract, direction, inv_direction, stp_price, qty=1):
    #Create order object
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = 'MKT'
    order.orderId = app.nextorderId
    app.nextorderId += 1
    order.eTradeOnly = ''
    order.firmQuoteOnly = ''
    order.transmit = False
    
    stop_order = Order()
    stop_order.action = inv_direction
    stop_order.totalQuantity = qty
    stop_order.orderType = 'STP'
    stop_order.auxPrice = stp_price
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    stop_order.eTradeOnly = ''
    stop_order.firmQuoteOnly = ''
    stop_order.transmit = True
    
  
    #submit order
    time.sleep(1)
    print('Placing {} order'.format(direction))
    app.placeOrder(order.orderId, contract, order)
    time.sleep(1)
    print('Placing STOP-LOSS order')
    app.placeOrder(stop_order.orderId, contract, stop_order)
    time.sleep(1)
    app.nextorderId += 1
    
    print('Full bracket order placed')
    print('Order placed at time: ' + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

def check_for_trade(df, contract):
    start_time = df.index[-1] - pd.Timedelta(minutes=3)
    df.index = pd.to_datetime(df.index, unit='m')
    
    data_open = df.resample('min').first()
    data_close = df.resample('min').last()
    data = df.copy()
    
    data = data.resample('min').mean()
    data['Open'] = data_open
    data['Close'] = data_close


    # Buy signal

    if (data['Open'][-1] >= prior_close_long):

        stp_p = round((data['Open'][-1] * 0.99), 0)


        print('Signal: Buy')
        print('Stop Loss: ' + str(stp_p))

        submit_order(contract, direction='BUY', inv_direction='SELL', stp_price=stp_p)
        return True



    if (data['Open'][-1] <= prior_close_short):

        stp_p = round((data['Open'][-1] * 1.01), 0)


        print('Signal: Sell')
        print('Stop Loss: ' + str(stp_p))

        submit_order(contract, direction='SELL', inv_direction='BUY', stp_price=stp_p)
        return True



#Main
app = IBapi()
app.nextorderId = None
app.connect('127.0.0.1', 7497, 124)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop)
api_thread.start()

#Check if the API is connected via orderid
while True:
    if isinstance(app.nextorderId, int):
        print('Connected')
        break
    else:
        print('Waiting for connection')
        time.sleep(1)

#Create contract object
NQ_contract = app.futures_contract(ticker)

#-----------------------------------------------------------#

#Request tick data for google using custom function
df = app.tick_df(401, NQ_contract)

print('Requesting tick data used for order conditions calculations')

#Verify data stream
time.sleep(10)
for i in range(100):
    if len(df) > 0:
        break
    
    time.sleep(0.3)

if i == 99:
    app.disconnect()
    raise Exception ('Error with Tick data stream')

print('Streaming tick data')

#Check if there is enough data
data_length = df.index[-1] - df.index[0]
if data_length.seconds < 10:
    time.sleep(10 - data_length.seconds)
    
print('Tick data with enough size to calculate EMAs')
print('Waiting for leverage-etf signal')
#Main loop
while True:
    check_for_trade(df, NQ_contract)
    break
    time.sleep(1)

app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Waiting for connection
The next valid order id is:  54
Connected
Requesting tick data used for order conditions calculations
Streaming tick data
Tick data with enough size to calculate EMAs
Waiting for leverage-etf signal
Signal: Buy
Stop Loss: 11527.0
Placing BUY order


ERROR 55 399 Order Message: SELL 1 MNQ DEC'22 Warning: your order will not be placed at the exchange until 2022-12-09 08:30:00 US/Central


Placing STOP-LOSS order
Full bracket order placed
Order placed at time: 2022-12-08 18:43:32


- New strat
- Optimization
- Storage structure
- Fix when error happens rerun code